In [ ]:
import numpy as np  
import geopandas as gpd  
import matplotlib.pyplot as plt  
import cartopy.crs as ccrs  
import xarray as xr  
from cartopy.mpl.gridliner import LATITUDE_FORMATTER, LONGITUDE_FORMATTER  
from scipy.interpolate import griddata  
from matplotlib.font_manager import FontProperties  
from shapely.geometry import Point  # Import Point class

# Load NetCDF data and shapefile
your_nc_data = xr.open_dataset("grace data/clip1_anjigrace.nc")  # Replace with actual NetCDF file path
your_shapefile = gpd.read_file("Export_Output.shp")              # Replace with actual shapefile path

# Specify time index
time_index = 11  # Change as needed

# Convert data to DataFrame and calculate mean
spatial_df = your_nc_data.to_dataframe().reset_index()
spatial_df['mean_variable'] = spatial_df[['your variable']].mean(axis=1)  ### Replace 'your variable' and 'mean_variable' with your variable name ### 

# Group by latitude and longitude
averaged_df = spatial_df.groupby(['lat', 'lon']).mean().reset_index()

# Extract averaged values
latitudes = averaged_df['lat'].values  
longitudes = averaged_df['lon'].values  
avg_values = averaged_df['mean_variable'].values                        

min_lon, min_lat, max_lon, max_lat = your_shapefile.total_bounds 

# Create mesh grid
num_lon, num_lat = 500, 500  
grid_lons = np.linspace(min_lon, max_lon, num_lon)  
grid_lats = np.linspace(min_lat, max_lat, num_lat)  
grid_lons, grid_lats = np.meshgrid(grid_lons, grid_lats)  

# Interpolate variable
interpolated_variable = griddata((longitudes, latitudes), avg_values, (grid_lons, grid_lats), method='linear')

# Create PlateCarree projection for mapping
projection = ccrs.PlateCarree()

# Create a mask for points within the shapefile
mask = np.zeros_like(interpolated_variable, dtype=bool)  
points = np.vstack([grid_lons.ravel(), grid_lats.ravel()]).T 
for geom in your_shapefile.geometry:
    mask = mask | np.array([geom.contains(Point(x, y)) for x, y in points]).reshape(grid_lons.shape)  
interpolated_variable[~mask] = np.nan  

# Plotting
plt.figure(figsize=(8, 8))  ### Set figure size ###
ax = plt.axes(projection=projection)  

# Add gridlines
gridlines = ax.gridlines(draw_labels=True, color='gray', linestyle='--', linewidth=0.5, zorder=1)  
gridlines.top_labels = False  
gridlines.right_labels = False
gridlines.xlabel_style = {'size': 20, 'color': 'black', 'weight': 'bold'}  
gridlines.ylabel_style = {'size': 20, 'color': 'black', 'weight': 'bold'}  

# Plot the interpolated data
pcm = ax.pcolormesh(grid_lons, grid_lats, interpolated_variable, cmap='Spectral', transform=projection)  

# Add color bar
cbar = plt.colorbar(pcm, ax=ax, orientation='vertical', shrink=0.6)  
cbar.set_label('your variable', fontsize=20, fontweight='bold', color='black')   ### Replace 'your variable' with required colorbar label ###

# Customize color bar ticks
cbar.ax.yaxis.set_tick_params(labelsize=20, width=1.5, color='black', labelcolor='black')  
tick_font = FontProperties(weight='bold', size=20)  
cbar.ax.yaxis.set_ticklabels(cbar.ax.yaxis.get_ticklabels(), fontproperties=tick_font)  

# Add shapefile polygons
your_shapefile.plot(ax=ax, facecolor='none', edgecolor='black', zorder=3)  

# Set latitude and longitude tick labels
lat_step = 4  
lon_step = 7  
ax.set_xticks(np.arange(np.floor(min_lon), np.ceil(max_lon) + 1, lon_step))  
ax.set_yticks(np.arange(np.floor(min_lat), np.ceil(max_lat) + 1, lat_step))  
ax.set_xticklabels([f"{abs(int(x))}°{'E' if x > 0 else 'W'}" for x in ax.get_xticks()], fontsize=20, fontweight='bold', color='black')  
ax.set_yticklabels([f"{abs(int(y))}°{'N' if y > 0 else 'S'}" for y in ax.get_yticks()], fontsize=20, fontweight='bold', color='black')  

# Set the plot extent
ax.set_extent([min_lon, max_lon, min_lat, max_lat], crs=projection)  

# Save the figure
plt.savefig('output_plot.tiff', dpi=300, bbox_inches='tight', facecolor='white')  
plt.show()  # Display the plot
